In [1]:
import os
from itertools import islice

import numpy as np
import pandas as pd

from skimage.external import tifffile
from scipy.ndimage import gaussian_filter
from skimage.external.tifffile import imread
from scipy.ndimage.measurements import center_of_mass

import ipywidgets as widgets

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

In [2]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'

In [3]:
myFile= r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\sel_cells_20_50_10_newctrl.csv'

In [4]:
myBigData=pd.read_csv(myFile)

In [7]:
myBigData=myBigData.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'])

In [8]:
myBigData.head()

,file,original cell name,Amy_mod,movie,x,y,ab1 (heterochromatin),ab1 ch,ab2 (MCM),ab2 ch,...,random_10perc_ch1,hetChrom_10perc_ch2,random_10perc_ch2,hetChrom_10perc_ch3,random_10perc_ch3,exp_folder,hetChrom_newctl_volume,hetChrom_newctl_ch1,hetChrom_newctl_ch2,hetChrom_newctl_ch3
0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172,956,HP1B,1,MCM3,2,...,281528813.0,174067475.0,131640512.0,1.352231e+09,1.043517e+09,20190625-MSL rename,280370,5.739116e+09,3.726297e+09,9.123801e+09
1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158,906,HP1B,1,MCM3,2,...,266516067.0,162411790.0,128299406.0,1.353023e+09,1.065184e+09,20190625-MSL rename,250606,5.403750e+09,3.628343e+09,8.997863e+09
2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832,252,HP1B,1,MCM3,2,...,318905411.0,194622718.0,148176740.0,1.366670e+09,1.118365e+09,20190625-MSL rename,189925,6.729417e+09,4.442309e+09,1.076495e+10
3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888,160,HP1B,1,MCM3,2,...,285348983.0,195376713.0,139563277.0,1.298882e+09,1.038559e+09,20190625-MSL rename,220076,5.459073e+09,3.733122e+09,9.257708e+09
4,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172,324,HP1B,1,MCM3,2,...,698512214.0,248463390.0,183210463.0,2.583220e+09,2.158853e+09,20190625-MSL rename,436430,8.007695e+09,4.024894e+09,1.674243e+10


In [9]:
## not my code; from online open source
## https://www.programcreek.com/python/example/57102/numpy.ogrid

def make_3d_mask(img_shape, center, radius, shape='sphere'):
    mask = np.zeros(img_shape)
    radius = np.rint(radius)
    center = np.rint(center)
    sz = np.arange(int(max(center[0] - radius, 0)), int(max(min(center[0] + radius + 1, img_shape[0]), 0)))
    sy = np.arange(int(max(center[1] - radius, 0)), int(max(min(center[1] + radius + 1, img_shape[1]), 0)))
    sx = np.arange(int(max(center[2] - radius, 0)), int(max(min(center[2] + radius + 1, img_shape[2]), 0)))
    sz, sy, sx = np.meshgrid(sz, sy, sx)

    distance2 = ((center[0] - sz) ** 2
                 + (center[1] - sy) ** 2
                 + (center[2] - sx) ** 2)
    distance_matrix = np.ones_like(mask) * np.inf
    distance_matrix[sz, sy, sx] = distance2
    mask[(distance_matrix <= radius ** 2)] = 1
   
    return mask 

In [10]:
## modified the function above to return the 
def make_3d_mask_dimension(img_shape, center, radius, shape='sphere'):
    mask = np.zeros(img_shape)
    radius = np.rint(radius)
    center = np.rint(center)
    sz = np.arange(int(max(center[0] - radius, 0)), int(max(min(center[0] + radius + 1, img_shape[0]), 0)))
    sy = np.arange(int(max(center[1] - radius, 0)), int(max(min(center[1] + radius + 1, img_shape[1]), 0)))
    sx = np.arange(int(max(center[2] - radius, 0)), int(max(min(center[2] + radius + 1, img_shape[2]), 0)))
    sz, sy, sx = np.meshgrid(sz, sy, sx)
    
    distance2 = ((center[0] - sz) ** 2
                 + (center[1] - sy) ** 2
                 + (center[2] - sx) ** 2)
    distance_matrix = np.ones_like(mask) * np.inf
    distance_matrix[sz, sy, sx] = distance2
    mask[(distance_matrix <= radius ** 2)] = 1

    return distance_matrix <= radius ** 2 

In [153]:
i=0

# Progress Bar
progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(myBigData),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)

for i, cell in myBigData.iterrows():
    percentile = 20

    # open tiff image of a cell
    myImage=imread(cell.path)    
    myChannel = myImage[:, 0, :, :] #1st channel is hetChrom (as stained by HP1B or H3K9Me3)

    # open nucleus mask
    nucleusMaskPath=cell.path
    nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
    nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
    nucMask=imread(nucleusMaskPath)
    nucMask=nucMask.astype(bool)

    # open het mask - new control
    myHetMaskPath=cell.path
    myHetMaskPath=myHetMaskPath.replace('data_tiff',f'segmentation_{percentile}')
    myHetMaskPath=myHetMaskPath.replace('.tif','_hetChrom.tif')
    het=imread(myHetMaskPath)
    het=het.astype(bool)
        
    nucMask[nucMask==255]=1

    
    ## resetting the limits in the frames
    y_axis=np.argmax(nucMask,axis=2)
    t=label(y_axis>0)
    props=regionprops(t)
    prop=props[0].bbox

    z_min = prop[0]
    z_max = prop[2]
    
    total_frame=z_max-z_min
    while((z_max-z_min)/(total_frame) > 3/4):
        z_max-=1
        z_min+=1
    
    ## making sphere mask and outer mask
    radius=1
    sphMask=make_3d_mask(nucMask.shape, center_of_mass(nucMask), radius, shape='sphere')
    sphMask=sphMask.astype(bool)
    

    while(np.sum(sphMask)/np.sum(nucMask)<0.20):
        radius+=1
        sphMask=make_3d_mask(nucMask.shape, center_of_mass(nucMask), radius, shape='sphere')
        sphMask=sphMask.astype(bool)


    ## making edge mask and limiting both masks to the new dimensions
    sphMask[0:z_min,:,:]=0
    sphMask[z_max:, :,:]=0
    outerMask=nucMask.copy()
    outerMask[sphMask]=0
    outerMask[0:z_min,:,:]=0
    outerMask[z_max:, :,:]=0

    outerMask=outerMask.astype(bool)
        
    ## masking heterochromatin with the sphere mask
    sph=sphMask.copy()
    sph=sph.astype(bool)
    het_copied=het.copy()
    het_copied=het_copied.astype(bool)
    het_sphere_test=het_copied[sphMask]
    het_inner=ma.zeros(het.shape)
    het_inner[sphMask]=het_sphere_test
    
    ## masking het with outer mask
    outer=outerMask.copy()
    outer=outer.astype(bool)
    het_copied2=het.copy()
    het_copied2=het_copied2.astype(bool)
    het_outer=ma.zeros(het.shape)
    het_outer_mask=het_copied2[outer]
    het_outer[outer]=het_outer_mask

    
    ## saving created masks
    newSegFolder = os.path.dirname(cell.path).replace('data_tiff','circular_method')

    try:
        os.mkdir(newSegFolder)
    except:
        pass

    maskFile=os.path.basename(cell.path).replace('.tif',f'_inner_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), sphMask.astype('uint8'))
    
    maskFile=os.path.basename(cell.path).replace('.tif',f'_outer_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), outerMask.astype('uint8'))

    maskFile = os.path.basename(cell.path).replace('.tif',f'_het_inner_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), het_inner.astype('uint8'))
    
    maskFile = os.path.basename(cell.path).replace('.tif',f'_het_outer_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), het_outer.astype('uint8'))
    
    # change the masked image to binary
    het_outer=het_outer.astype(bool)
    het_inner=het_inner.astype(bool)
    
    
    # calculate signals - HET 
    df = pd.DataFrame(columns = ['nucleus_otsu_sphere_inner_volume', 'nucleus_otsu_sphere_outer_volume',
        f'HetChrom_sphere_inner_{percentile}_Volume',f'HetChrom_sphere_inner_{percentile}_ch1',f'HetChrom_sphere_inner_{percentile}_ch2',f'HetChrom_sphere_inner_{percentile}_ch3',
        f'HetChrom_sphere_outer_{percentile}_Volume',f'HetChrom_sphere_outer_{percentile}_ch1',f'HetChrom_sphere_outer_{percentile}_ch2',f'HetChrom_sphere_outer_{percentile}_ch3'])

    df.Slice = range(myChannel.shape[0])
    df.HetChrom_sphere_inner_20_Volume =  het_inner.sum(-1).sum(-1)
    df.HetChrom_sphere_outer_20_Volume =  het_outer.sum(-1).sum(-1)

    for ch in range(3):
        # inner mask
        currCh=np.copy(myImage[:,ch,:,:])
        currCh[~het_inner] = 0
        signal = currCh.sum(-1).sum(-1)
        exec(f'df.HetChrom_sphere_inner_20_ch{ch+1}=signal')
        #exec(f"myBigData.loc[i,'hetChrom_sphere_inner_{percentile}_ch{ch+1}']=np.sum(signal)")

        
        # outer mask
        outerch=np.copy(myImage[:,ch,:,:])
        outerch[~het_outer] = 0
        signal = outerch.sum(-1).sum(-1)
        exec(f'df.HetChrom_sphere_outer_20_ch{ch+1}=signal')
        #exec(f"myBigData.loc[i,'hetChrom_sphere_outer_{percentile}_ch{ch+1}']=np.sum(signal)")

        
  
    # save signals
    dfSavePath=cell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_hetChrom_sphere_{percentile}_updated.csv')
    df.to_csv(dfSavePath)
    
    # add info to cell info file
    myBigData.loc[i,f'hetChrom_sphere_inner_{percentile}_volume']=np.sum(df.HetChrom_sphere_inner_20_Volume)
    myBigData.loc[i,f'hetChrom_sphere_outer_{percentile}_volume']=np.sum(df.HetChrom_sphere_outer_20_Volume)
    myBigData.loc[i,'nucleus_otsu_sphere_inner_volume']=np.sum(df.nucleus_otsu_sphere_inner_volume)
    myBigData.loc[i,'nucleus_otsu_sphere_outer_volume']=np.sum(df.nucleus_otsu_sphere_outer_volume)
    for ch in range(3):
        exec(f"myBigData.loc[i,'hetChrom_sphere_inner_{percentile}_ch{ch+1}']=np.sum(df.HetChrom_sphere_inner_20_ch{ch+1})") 
        exec(f"myBigData.loc[i,'hetChrom_sphere_outer_{percentile}_ch{ch+1}']=np.sum(df.HetChrom_sphere_outer_20_ch{ch+1})")

    # calculate signals - DAPI
    df = pd.DataFrame(columns = ['nucleus_Otsu_inner_volume','nucleus_Otsu_inner_ch1',
                                 'nucleus_Otsu_inner_ch2','nucleus_Otsu_inner_ch3',
                                'nucleus_Otsu_outer_volume','nucleus_Otsu_outer_ch1',
                                 'nucleus_Otsu_outer_ch2','nucleus_Otsu_outer_ch3'])
    
    
    # make nucleus mask binary
    nucMask = nucMask.astype(bool)

    # open tiff image of a cell for signal calculations
    myImage=imread(cell.path)    
    ch1 = myImage[:, 0, :, :] #1st channel - hetChrom (as stained by HP1B or H3K9Me3)
    ch2 = myImage[:, 1, :, :] #2nd channel - MCM
    ch3 = myImage[:, 2, :, :] #3rd channel - DAPI
    ch1_1=ch1.copy()
    ch2_1=ch2.copy()
    ch3_1=ch3.copy()
   
    # real calculations
    masks=['sphMask','outerMask']
    masksNames=['nucleus_Otsu_inner', 'nucleus_Otsu_outer']
    signals=[None,'ch1','ch2','ch3']
    copied_signals=[None,'ch1_1','ch2_1','ch3_1']
    signalsNames=['volume','ch1','ch2','ch3']
    
    df.Slice = range(ch1.shape[0])
    
    for myMask,myMaskName in zip(masks,masksNames):
        
        for mySignal,mySignalName,realSig in zip(signals,signalsNames, copied_signals):
            ch1=ch1_1.copy()
            ch2=ch2_1.copy()
            ch3=ch3_1.copy()
            if mySignal is None:
                mySignal=myMask
                
            myString=f'{mySignal}[~{myMask}]=0'
           
            exec(myString)
            myString=f'df.{myMaskName}_{mySignalName}={mySignal}.sum(-1).sum(-1)'
            exec(myString)
            

    
    # save signals (per frame)
    dfSavePath=cell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_nuc_otsu_radial_updated.csv')
    df.to_csv(dfSavePath)

    # add info to cell info file
    for myMeasurement in df.columns:
        
        myString=f"myBigData.loc[i,'{myMeasurement}']=np.sum(df.{myMeasurement})"
        exec(myString)
    
    
    
    progBar.value=progBar.value+1
    

IntProgress(value=0, description='Progress:', max=300)

C:\ProgramData\Anaconda3_reinstalled\lib\site-packages\ipykernel_launcher.py:128: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\ProgramData\Anaconda3_reinstalled\lib\site-packages\ipykernel_launcher.py:187: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [154]:
myBigData.to_csv( r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\signal_data_total_v1.csv')

# modifying the code so that the edge mask would include the entire rest of the nucleus not end when sphere ends (1/24)

In [11]:
i=0

# Progress Bar
progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(myBigData),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)

for i, cell in myBigData.iterrows():
    percentile = 20

    # open tiff image of a cell
    myImage=imread(cell.path)    
    myChannel = myImage[:, 0, :, :] #1st channel is hetChrom (as stained by HP1B or H3K9Me3)

    # open nucleus mask
    nucleusMaskPath=cell.path
    nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
    nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
    nucMask=imread(nucleusMaskPath)
    nucMask=nucMask.astype(bool)

    # open het mask - new control
    myHetMaskPath=cell.path
    myHetMaskPath=myHetMaskPath.replace('data_tiff',f'segmentation_{percentile}')
    myHetMaskPath=myHetMaskPath.replace('.tif','_hetChrom.tif')
    het=imread(myHetMaskPath)
    het=het.astype(bool)
        
    nucMask[nucMask==255]=1

    
    ## resetting the limits in the frames
    y_axis=np.argmax(nucMask,axis=2)
    t=label(y_axis>0)
    props=regionprops(t)
    prop=props[0].bbox

    z_min = prop[0]
    z_max = prop[2]
    
    total_frame=z_max-z_min
    while((z_max-z_min)/(total_frame) > 3/4):
        z_max-=1
        z_min+=1
    
    ## making sphere mask and outer mask
    radius=1
    sphMask=make_3d_mask(nucMask.shape, center_of_mass(nucMask), radius, shape='sphere')
    sphMask=sphMask.astype(bool)
    

    while(np.sum(sphMask)/np.sum(nucMask)<0.20):
        radius+=1
        sphMask=make_3d_mask(nucMask.shape, center_of_mass(nucMask), radius, shape='sphere')
        sphMask=sphMask.astype(bool)


    ## making edge mask and limiting both masks to the new dimensions
    sphMask[0:z_min,:,:]=0
    sphMask[z_max:, :,:]=0
    outerMask=nucMask.copy()
    outerMask[sphMask]=0
    #outerMask[0:z_min,:,:]=0
    #outerMask[z_max:, :,:]=0

    outerMask=outerMask.astype(bool)
        
    ## masking heterochromatin with the sphere mask
    sph=sphMask.copy()
    sph=sph.astype(bool)
    het_copied=het.copy()
    het_copied=het_copied.astype(bool)
    het_sphere_test=het_copied[sphMask]
    het_inner=ma.zeros(het.shape)
    het_inner[sphMask]=het_sphere_test
    
    ## masking het with outer mask
    outer=outerMask.copy()
    outer=outer.astype(bool)
    het_copied2=het.copy()
    het_copied2=het_copied2.astype(bool)
    het_outer=ma.zeros(het.shape)
    het_outer_mask=het_copied2[outer]
    het_outer[outer]=het_outer_mask

    
    ## saving created masks
    newSegFolder = os.path.dirname(cell.path).replace('data_tiff','circular_method')

    try:
        os.mkdir(newSegFolder)
    except:
        pass

    maskFile=os.path.basename(cell.path).replace('.tif',f'_inner_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), sphMask.astype('uint8'))
    
    maskFile=os.path.basename(cell.path).replace('.tif',f'_outer_v2{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), outerMask.astype('uint8'))

    maskFile = os.path.basename(cell.path).replace('.tif',f'_het_inner_{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), het_inner.astype('uint8'))
    
    maskFile = os.path.basename(cell.path).replace('.tif',f'_het_outer_v2{percentile}.tif')
    tifffile.imsave(os.path.join(newSegFolder,maskFile), het_outer.astype('uint8'))
    
    # change the masked image to binary
    het_outer=het_outer.astype(bool)
    het_inner=het_inner.astype(bool)
    
    
    # calculate signals - HET 
    df = pd.DataFrame(columns = ['nucleus_otsu_sphere_inner_volume', 'nucleus_otsu_sphere_outer_volume',
        f'HetChrom_sphere_inner_{percentile}_Volume',f'HetChrom_sphere_inner_{percentile}_ch1',f'HetChrom_sphere_inner_{percentile}_ch2',f'HetChrom_sphere_inner_{percentile}_ch3',
        f'HetChrom_sphere_outer_{percentile}_Volume',f'HetChrom_sphere_outer_{percentile}_ch1',f'HetChrom_sphere_outer_{percentile}_ch2',f'HetChrom_sphere_outer_{percentile}_ch3'])

    df.Slice = range(myChannel.shape[0])
    df.HetChrom_sphere_inner_20_Volume =  het_inner.sum(-1).sum(-1)
    df.HetChrom_sphere_outer_20_Volume =  het_outer.sum(-1).sum(-1)

    for ch in range(3):
        # inner mask
        currCh=np.copy(myImage[:,ch,:,:])
        currCh[~het_inner] = 0
        signal = currCh.sum(-1).sum(-1)
        exec(f'df.HetChrom_sphere_inner_20_ch{ch+1}=signal')
        #exec(f"myBigData.loc[i,'hetChrom_sphere_inner_{percentile}_ch{ch+1}']=np.sum(signal)")

        
        # outer mask
        outerch=np.copy(myImage[:,ch,:,:])
        outerch[~het_outer] = 0
        signal = outerch.sum(-1).sum(-1)
        exec(f'df.HetChrom_sphere_outer_20_ch{ch+1}=signal')
        #exec(f"myBigData.loc[i,'hetChrom_sphere_outer_{percentile}_ch{ch+1}']=np.sum(signal)")

        
  
    # save signals
    dfSavePath=cell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_hetChrom_sphere_{percentile}_updated.csv')
    df.to_csv(dfSavePath)
    
    # add info to cell info file
    myBigData.loc[i,f'hetChrom_sphere_inner_{percentile}_volume']=np.sum(df.HetChrom_sphere_inner_20_Volume)
    myBigData.loc[i,f'hetChrom_sphere_outer_{percentile}_volume']=np.sum(df.HetChrom_sphere_outer_20_Volume)
    myBigData.loc[i,'nucleus_otsu_sphere_inner_volume']=np.sum(df.nucleus_otsu_sphere_inner_volume)
    myBigData.loc[i,'nucleus_otsu_sphere_outer_volume']=np.sum(df.nucleus_otsu_sphere_outer_volume)
    for ch in range(3):
        exec(f"myBigData.loc[i,'hetChrom_sphere_inner_{percentile}_ch{ch+1}']=np.sum(df.HetChrom_sphere_inner_20_ch{ch+1})") 
        exec(f"myBigData.loc[i,'hetChrom_sphere_outer_{percentile}_ch{ch+1}']=np.sum(df.HetChrom_sphere_outer_20_ch{ch+1})")

    # calculate signals - DAPI
    df = pd.DataFrame(columns = ['nucleus_Otsu_inner_volume','nucleus_Otsu_inner_ch1',
                                 'nucleus_Otsu_inner_ch2','nucleus_Otsu_inner_ch3',
                                'nucleus_Otsu_outer_volume','nucleus_Otsu_outer_ch1',
                                 'nucleus_Otsu_outer_ch2','nucleus_Otsu_outer_ch3'])
    
    
    # make nucleus mask binary
    nucMask = nucMask.astype(bool)

    # open tiff image of a cell for signal calculations
    myImage=imread(cell.path)    
    ch1 = myImage[:, 0, :, :] #1st channel - hetChrom (as stained by HP1B or H3K9Me3)
    ch2 = myImage[:, 1, :, :] #2nd channel - MCM
    ch3 = myImage[:, 2, :, :] #3rd channel - DAPI
    ch1_1=ch1.copy()
    ch2_1=ch2.copy()
    ch3_1=ch3.copy()
   
    # real calculations
    masks=['sphMask','outerMask']
    masksNames=['nucleus_Otsu_inner', 'nucleus_Otsu_outer']
    signals=[None,'ch1','ch2','ch3']
    copied_signals=[None,'ch1_1','ch2_1','ch3_1']
    signalsNames=['volume','ch1','ch2','ch3']
    
    df.Slice = range(ch1.shape[0])
    
    for myMask,myMaskName in zip(masks,masksNames):
        
        for mySignal,mySignalName,realSig in zip(signals,signalsNames, copied_signals):
            ch1=ch1_1.copy()
            ch2=ch2_1.copy()
            ch3=ch3_1.copy()
            if mySignal is None:
                mySignal=myMask
                
            myString=f'{mySignal}[~{myMask}]=0'
           
            exec(myString)
            myString=f'df.{myMaskName}_{mySignalName}={mySignal}.sum(-1).sum(-1)'
            exec(myString)
            

    
    # save signals (per frame)
    dfSavePath=cell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_nuc_otsu_radial_updated.csv')
    df.to_csv(dfSavePath)

    # add info to cell info file
    for myMeasurement in df.columns:
        
        myString=f"myBigData.loc[i,'{myMeasurement}']=np.sum(df.{myMeasurement})"
        exec(myString)
    
    
    
    break
    progBar.value=progBar.value+1
    

IntProgress(value=0, description='Progress:', max=300)

C:\ProgramData\Anaconda3_reinstalled\lib\site-packages\ipykernel_launcher.py:123: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\ProgramData\Anaconda3_reinstalled\lib\site-packages\ipykernel_launcher.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
